###Set working directory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Face_Recognition_with_SVM
# !ls

/content/drive/MyDrive/Colab Notebooks/Face_Recognition_with_SVM


###Import essential libraries

In [3]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

###Obtain image feature & label

In [4]:
folder='a1_TrainingPhotos/' # location of training images
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image

    img=img.resize([32,64])  # resize image  to a fixed size

    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True,channel_axis=-1) # obtain HOG features

    X.append(fd)

    # obtain training labels from file name

    if re.match("Amir*", filename):
      Yt=0
    elif re.match("Jaya*", filename):
      Yt=1
    elif re.match("Hir*", filename):
      Yt=2
    elif re.match("Anupam*", filename):
      Yt=3
    else:
      Yt=4

    Y.append(Yt)

    i=i+1


In [5]:
fd.shape

(3780,)

In [6]:
target_names=['AamirKhan', 'JayaBadhuri', 'HritikRoshan', 'AnupamKher']

###Split data into training/test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###Train SVM classifier

In [8]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.001, probability=True)


In [9]:
# Save best SVM classifier weights for use at prediction

import joblib
joblib.dump(clf.best_estimator_, 'c2_FR_svm_classifier.pkl', compress = 1) # Only best parameters

['c2_FR_svm_classifier.pkl']

###Model performance on test set

In [10]:
y_pred = clf.predict(X_test)
y_pred

array([0, 1, 0, 0, 0, 2, 3, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 1, 2, 0, 1, 0, 2, 1, 1, 0, 1, 1,
       2, 0, 0, 1, 0, 2, 3, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 0, 0, 0, 2, 1,
       0, 2, 2, 1, 0, 1, 1, 0, 1, 0, 1, 3, 0, 1, 1, 0, 1, 0, 2, 0, 1, 3,
       0, 1, 2, 0, 0, 2, 1, 0, 1, 1, 0, 2, 0, 1, 0, 2, 1, 0, 0])

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.84      0.74        38
           1       0.94      0.82      0.88        40
           2       0.74      0.67      0.70        21
           3       0.50      0.25      0.33         8

    accuracy                           0.76       107
   macro avg       0.71      0.65      0.66       107
weighted avg       0.77      0.76      0.75       107



###Coding ends here!